# LangChain Conversation History Management
In the [LangChain Expression Language (LCEL)](./lcel.ipynb), we covered LCEL at a high level, demonstrating specifically how to chain a prompt engineered chat prompt with an LLM, namely MLX. What I failed to demonstrate in that notebook was how to think about memory (aka chat conversation management), and to be honest, I underestimated how "involved" of a thing this got to be! 😅 To be clear, what we will be covering in this notebook is less of a technical concern and more of a business logic concern.

While LangChain offers many mechanisms for handling chat conversations (aka memory) correctly, I found some of the higher level ones to not be satisfactory for our purposes. Specifically, since I want us to adhere to a fixed schema, the high level abstraction objects provided by LangChain simply don't operate in the ideal way in which we need them to. No worries! We can still work around this without having to abandon LangChain. We're just going to need to do some special stuff throughout this notebook!

## High Level Flow
Before we get into the code itself, let's talk about how we want to think about the flow. For simplicity's sake, we are going to be ultimately saving this chat history as a JSON file. This JSON file should look like the schema that we've defined in the file `data/schema.json`.

Let's say that the user is loading the MLX Gradio UI interface, either for the first time ever or as a returning user. Here is the flow of how we should be thinking about our data:

1. **Loading the conversation history from file**: Just as it sounds, we will want to load the conversation history from file so that the user can interact with their historical conversations if they would like. Now, it's possible that this is the user's first time interacting with the chatbot, so it may be that we need to create this file from scratch!
2. **Setting a new conversation ID**: Regardless if the user is new or returning, we are going to make the assumption that the user will want to begin with a new conversation. This means that we will need to instantiate a new conversation ID so that we can keep appending new conversation interactions to that same conversation thread.
3. **Managing conversation back-and-forth**: As the conversation proceeds, we will want to continually update our conversation schema with any new human and AI interactions. This will include also autosaving them to file for the user's convenience.
4. **Starting a new conversation / loading an existing conversation**: At any point, the user may want to pivot from their current conversation to either a new conversation or to continue another historical conversation loaded from our file as part of step 1. If this is the case, we will need to ensure that our backend system is referencing the correct conversation interaction.

## Notebook Setup
In this section, we'll do all our usual set ups. We'll also set up the LangChain prompt structure and model. All these are things we've already explored in other notebooks.

In [1]:
# Importing the necessary Python libraries
import os
import json
import uuid
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.llms.mlx_pipeline import MLXPipeline
from langchain_community.chat_models.mlx import ChatMLX

In [2]:
# Setting a default system prompt
DEFAULT_SYSTEM_PROMPT = 'You are a helpful assistant.'

In [3]:
# Setting up the Chat prompt template
system_message_prompt = SystemMessagePromptTemplate.from_template(template = DEFAULT_SYSTEM_PROMPT)
human_message_prompt = HumanMessagePromptTemplate.from_template(template = "{input}")
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [4]:
# Setting constant values to represent model name and directory
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
BASE_DIRECTORY = '../models'
MLX_DIRECTORY = f'{BASE_DIRECTORY}/mlx'
mlx_model_directory = f'{MLX_DIRECTORY}/{MODEL_NAME}'

In [5]:
# Setting up the LangChain MLX LLM
llm = MLXPipeline.from_model_id(
    model_id = mlx_model_directory,
    pipeline_kwargs = {
        'temp': 0.7,
        'max_tokens': 1000
    }
)

# Setting up the LangChain MLX Chat Model with the LLM above
chat_model = ChatMLX(llm = llm)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Starting the Conversation History Schema
As mentioned before, we are going to be emulating the structure of the schema as defined in `data/schema.json`. In this notebook, we are going to pretend as if the user is a brand new user, so we will need to set up the conversation history schema from scratch.

In [11]:
# Generating a current conversation ID
current_conversation_id = str(uuid.uuid4())
current_conversation_id

'c0393de1-2a2f-4e6a-80d0-cee64fcc6b60'

In [10]:
# Creating the base conversation history schema per a single user
BASE_CONVERSATION_HISTORY_SCHEMA = {
    'user_id': 'default_username',
    'conversation_history': [
        {
            'conversation_id': current_conversation_id,
            'summary_title': '',
            'system_prompt': DEFAULT_SYSTEM_PROMPT,
            'conversation': []
        }
    ]
}

BASE_CONVERSATION_HISTORY_SCHEMA

{'user_id': 'default_username',
 'conversation_history': [{'conversation_id': '6c1bdf13-f669-45d4-a2bd-e2ac6cb19e79',
   'summary_title': '',
   'system_prompt': 'You are a helpful assistant.',
   'conversation': []}]}

In [8]:
BASE_CONVERSATION_HISTORY_SCHEMA

{'user_id': 'default_username',
 'conversation_history': [{'conversation_id': '6c1bdf13-f669-45d4-a2bd-e2ac6cb19e79',
   'summary_title': '',
   'system_prompt': 'You are a helpful assistant.',
   'conversation': []}]}